L'objectif de ce notebook est de créer le dataset contenant les images satellites (1km x 1km) et l'intensité lumineuse correspondante à cette zone.

In [ ]:
import pandas as pd
import numpy as np
from simpledbf import Dbf5
from osgeo import gdal, ogr, osr
import seaborn as sns
import cv2
import urllib.request
import matplotlib.pyplot as plt

In [ ]:
folder_path = './image-nightlight_dataset'

In [ ]:
df_rwanda = pd.read_csv('./uganda')
df_rwanda = df_rwanda[['latitude', 'longitude']]
nb_lines = df_rwanda.shape[0]
print(nb_lines)
df_rwanda.head()

In [ ]:
night_file = './nightlights_intensities/F182010.v4d_web.stable_lights.avg_vis.tif'

dataset = gdal.Open(night_file, gdal.GA_ReadOnly)

print("Driver: {}/{}".format(dataset.GetDriver().ShortName,
                             dataset.GetDriver().LongName))
print("Size is {} x {} x {}".format(dataset.RasterXSize,
                                    dataset.RasterYSize,
                                    dataset.RasterCount))
print("Projection is {}".format(dataset.GetProjection()))
(upper_left_x, x_size, x_rotation, upper_left_y, y_rotation, y_size) = dataset.GetGeoTransform()
print("Origin = ({}, {})".format(upper_left_x, upper_left_y))
print("Pixel Size = ({}, {})".format(x_size, y_size))

band = dataset.GetRasterBand(1)
print("Band Type={}".format(gdal.GetDataTypeName(band.DataType)))
      
min = band.GetMinimum()
max = band.GetMaximum()
if not min or not max:
    (min,max) = band.ComputeRasterMinMax(True)
print("Min={:.3f}, Max={:.3f}".format(min,max))

band_array = band.ReadAsArray()
rows, cols = np.shape(band_array)

x_size = 1.0 / int(round(1 / float(x_size)))
y_size = - x_size
y_index = np.arange(band_array.shape[0])
x_index = np.arange(band_array.shape[1])
top_left_x_coords = upper_left_x + x_index * x_size
top_left_y_coords = upper_left_y + y_index * y_size
centroid_x_coords = top_left_x_coords + (x_size / 2)
centroid_y_coords = top_left_y_coords + (y_size / 2)

In [ ]:
def get_cell_idx(lon, lat, top_left_x_coords, top_left_y_coords):
    lon_idx = np.where(top_left_x_coords < lon)[0][-1]
    lat_idx = np.where(top_left_y_coords > lat)[0][-1]
    return lon_idx, lat_idx

def get_nightlight_feature(sample, band_array):
    x, y = sample
    lon_idx, lat_idx = get_cell_idx(x, y, top_left_x_coords, top_left_y_coords)
    
    # Select the 10 * 10 pixels
    left_idx = lon_idx - 5
    right_idx = lon_idx + 4
    up_idx = lat_idx - 5
    low_idx = lat_idx + 4
    luminosity_100 = []
    
    for i in range(left_idx, right_idx + 1):
        for j in range(up_idx, low_idx + 1):
            # Get the luminosity of this pixel
            luminosity = band_array[j, i]
            luminosity_100.append(luminosity)
    
    luminosity_100 = np.asarray(luminosity_100)
    
    max_ = np.max(luminosity_100)
    min_ = np.min(luminosity_100)
    mean_ = np.mean(luminosity_100)
    median_ = np.median(luminosity_100)
    std_ = np.std(luminosity_100)
    
    return band_array[lat_idx, lon_idx]

In [ ]:
ids, lats, longs = [], [], []

l = 0
for k in range(nb_lines):
    for i in range(10):
        for j in range(10):
            lats.append(df_rwanda['latitude'][k] - 5 * 0.009 + i * 0.009)
            longs.append(df_rwanda['longitude'][k] - 5 * 0.018 + j * 0.018)
            ids.append(l)
            l += 1

In [ ]:
mean, median, std = [], [], []
for m in range(len(lats)):
    sample = (longs[m], lats[m])
    m, md, st = get_nightlight_feature(sample, band_array)
    mean.append(m)
    median.append(md)
    std.append(st)

In [ ]:
data = {'id':ids, 'latitude':lats, 'longitude':longs, 'mean':mean, 'median':median, 'std':std}
df = pd.DataFrame(data)

In [ ]:
intensities = []
for m in range(len(lats)):
    sample = (longs[m], lats[m])
    intensities.append(get_nightlight_feature(sample, band_array))
    
data = {'id':ids, 'latitude':lats, 'longitude':longs, 'intensity':intensities}
df = pd.DataFrame(data)

In [ ]:
df.head()

In [ ]:
df.to_csv('./image-nightlight_data')

In [ ]:
df = pd.read_csv('./image-nightlight_data')
df = df[['id', 'intensity', 'latitude', 'longitude']]
df.head()

In [ ]:
api_key = ""

In [ ]:
coord = df.as_matrix()
print(np.shape(coord))
print(coord[0])
n = coord.shape[0]
print(n)

In [ ]:
import time

In [ ]:
for i in range(3642, n):
    print('Step', i)
    
    clust, lat, long = coord[i][0], coord[i][2], coord[i][3]
    
    url = 'https://maps.googleapis.com/maps/api/staticmap?center='+str(lat)+','+str(long) + \
    '&zoom=16'+'&size=400x400'+'&maptype=satellite'+'&key='+str(api_key)
    print(url)
    
    filename = './image-nightlight_dataset/' + str(clust) + '.png'
    print(filename)
    
    urllib.request.urlretrieve(url, filename)
    
    time.sleep(0.2)